In [19]:
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

t = time()

enc_dict = {'Single': '1B','Double': '2B','Triple': '3B','Home Run': 'HR','Walk': 'BB',
            'Intent Walk': 'IBB','Hit By Pitch': 'HBP','Strikeout': 'K','Sac Fly': 'SF',
            'Grounded Into DP': 'GIDP','Groundout': 'GO','Lineout': 'LO','Pop Out': 'PO',
            'Flyout': 'FO','Fielders Choice': 'FC','Sac Bunt': 'SAC','Double Play': 'DP',
            'Triple Play': 'TP','Batter Interference': 'BI','Fan interference': 'FI',
            'Catcher Interference': 'CI','Field Error': 'ROE','Bunt Groundout': 'BGO',
            'Bunt Lineout': 'BLO','Bunt Pop Out': 'BPO','Fielders Choice Out': 'FCO',
            'Forceout': 'FORCE','Sacrifice Bunt DP': 'SBDP','Strikeout - DP': 'KDP',
            'Runner Out': 'RO','Sac Fly DP': 'SFDP'
           }

In [20]:
pitches = pd.read_csv('pitches.csv')
pitches.head()

px     pz  start_speed  end_speed  spin_rate  spin_dir  break_angle  \
0  0.416  2.963         92.9       84.1   2305.052   159.235        -25.0   
1 -0.191  2.347         92.8       84.1   2689.935   151.402        -40.7   
2 -0.518  3.284         94.1       85.2   2647.972   145.125        -43.7   
3 -0.641  1.221         91.0       84.0   1289.590   169.751         -1.3   
4 -1.821  2.083         75.4       69.6   1374.569   280.671         18.4   

   break_length  break_y      ax      ay      az  sz_bot  sz_top  \
0           3.2     23.7   7.665  34.685 -11.960    1.72    3.56   
1           3.4     23.7  12.043  34.225 -10.085    1.72    3.56   
2           3.7     23.7  14.368  35.276 -11.560    1.72    3.56   
3           5.0     23.8   2.104  28.354 -20.540    1.74    3.35   
4          12.0     23.8 -10.280  21.774 -34.111    1.72    3.56   

   type_confidence    vx0      vy0    vz0           x     x0           y  \
0              2.0 -6.409 -136.065 -3.995  101.140000  2.280  158.780000   
1              2.0 -8.411 -135.690 -5.980  124.280000  2.119  175.410000   
2              2.0 -9.802 -137.668 -3.337  136.740000  2.127  150.110000   
3              2.0 -8.071 -133.005 -6.567  109.685636  2.279  187.463482   
4              2.0 -6.309 -110.409  0.325  146.527525  2.179  177.242829   

     y0     z0  pfx_x  pfx_z  nasty  zone code type pitch_type  event_num  \
0  50.0  5.302   4.16  10.93   55.0   3.0    C    S         FF          3   
1  50.0  5.307   6.57  12.00   31.0   5.0    S    S         FF          4   
2  50.0  5.313   7.61  10.88   49.0   1.0    F    S         FF          5   
3  50.0  5.210   1.17   6.45   41.0  13.0    B    B         FF          6   
4  50.0  5.557  -8.43  -1.65   18.0  13.0    B    B         CU          7   

   b_score         ab_id  b_count  s_count  outs  pitch_num  on_1b  on_2b  \
0      0.0  2.015000e+09      0.0      0.0   0.0        1.0    0.0    0.0   
1      0.0  2.015000e+09      0.0      1.0   0.0        2.0    0.0    0.0   
2      0.0  2.015000e+09      0.0      2.0   0.0        3.0    0.0    0.0   
3      0.0  2.015000e+09      0.0      2.0   0.0        4.0    0.0    0.0   
4      0.0  2.015000e+09      1.0      2.0   0.0        5.0    0.0    0.0   

   on_3b  
0    0.0  
1    0.0  
2    0.0  
3    0.0  
4    0.0

In [21]:
games = pd.read_csv('games.csv')
games.head()

attendance  away_final_score away_team        date  elapsed_time  \
0       35055                 3       sln  2015-04-05           184   
1       45909                 1       ana  2015-04-06           153   
2       36969                 2       atl  2015-04-06           156   
3       31042                 6       bal  2015-04-06           181   
4       45549                 8       bos  2015-04-06           181   

        g_id  home_final_score home_team start_time    umpire_1B  \
0  201500001                 0       chn    7:17 PM  Mark Wegner   
1  201500002                 4       sea    1:12 PM    Ron Kulpa   
2  201500003                 1       mia    4:22 PM     Laz Diaz   
3  201500004                 2       tba    3:12 PM    Ed Hickox   
4  201500005                 0       phi    3:08 PM   Phil Cuzzi   

        umpire_2B        umpire_3B      umpire_HP          venue_name  \
0    Marty Foster  Mike Muchlinski   Mike Winters       Wrigley Field   
1    Brian Knight    Vic Carapazza  Larry Vanover        Safeco Field   
2  Chris Guccione      Cory Blaser    Jeff Nelson        Marlins Park   
3     Paul Nauert   Mike Estabrook    Dana DeMuth     Tropicana Field   
4   Tony Randazzo      Will Little    Gerry Davis  Citizens Bank Park   

                     weather                wind  delay  
0          44 degrees, clear   7 mph, In from CF      0  
1         54 degrees, cloudy       1 mph, Varies      0  
2  80 degrees, partly cloudy  16 mph, In from CF     16  
3           72 degrees, dome         0 mph, None      0  
4  71 degrees, partly cloudy   11 mph, Out to RF      0

In [22]:
atbats = pd.read_csv('atbats.csv')
atbats.head()

ab_id  batter_id      event       g_id  inning  o  p_score p_throws  \
0  2015000001     572761  Groundout  201500001       1  1        0        L   
1  2015000002     518792     Double  201500001       1  1        0        L   
2  2015000003     407812     Single  201500001       1  1        0        L   
3  2015000004     425509  Strikeout  201500001       1  2        0        L   
4  2015000005     571431  Strikeout  201500001       1  3        0        L   

   pitcher_id stand   top  
0      452657     L  True  
1      452657     L  True  
2      452657     R  True  
3      452657     R  True  
4      452657     L  True

In [23]:
player_name = pd.read_csv('player_names.csv')
player_name.head()

id first_name   last_name
0  452657        Jon      Lester
1  425794       Adam  Wainwright
2  457435       Phil        Coke
3  435400      Jason       Motte
4  519166       Neil     Ramirez

In [24]:
games.dtypes

attendance           int64
away_final_score     int64
away_team           object
date                object
elapsed_time         int64
g_id                 int64
home_final_score     int64
home_team           object
start_time          object
umpire_1B           object
umpire_2B           object
umpire_3B           object
umpire_HP           object
venue_name          object
weather             object
wind                object
delay                int64
dtype: object

In [25]:
atbats.dtypes

ab_id          int64
batter_id      int64
event         object
g_id           int64
inning         int64
o              int64
p_score        int64
p_throws      object
pitcher_id     int64
stand         object
top             bool
dtype: object

In [26]:
pitches.dtypes

px                 float64
pz                 float64
start_speed        float64
end_speed          float64
spin_rate          float64
spin_dir           float64
break_angle        float64
break_length       float64
break_y            float64
ax                 float64
ay                 float64
az                 float64
sz_bot             float64
sz_top             float64
type_confidence    float64
vx0                float64
vy0                float64
vz0                float64
x                  float64
x0                 float64
y                  float64
y0                 float64
z0                 float64
pfx_x              float64
pfx_z              float64
nasty              float64
zone               float64
code                object
type                object
pitch_type          object
event_num            int64
b_score            float64
ab_id              float64
b_count            float64
s_count            float64
outs               float64
pitch_num          float64
o

In [27]:
pitches['ab_id'] = pitches['ab_id'].astype(int)

In [28]:
player_name.rename(columns={'id': 'batter_id'}, inplace=True)

In [29]:
new_df = pd.merge(pitches, atbats,  how='left', left_on='ab_id', right_on = 'ab_id')
new_df.head()

px     pz  start_speed  end_speed  spin_rate  spin_dir  break_angle  \
0  0.416  2.963         92.9       84.1   2305.052   159.235        -25.0   
1 -0.191  2.347         92.8       84.1   2689.935   151.402        -40.7   
2 -0.518  3.284         94.1       85.2   2647.972   145.125        -43.7   
3 -0.641  1.221         91.0       84.0   1289.590   169.751         -1.3   
4 -1.821  2.083         75.4       69.6   1374.569   280.671         18.4   

   break_length  break_y      ax      ay      az  sz_bot  sz_top  \
0           3.2     23.7   7.665  34.685 -11.960    1.72    3.56   
1           3.4     23.7  12.043  34.225 -10.085    1.72    3.56   
2           3.7     23.7  14.368  35.276 -11.560    1.72    3.56   
3           5.0     23.8   2.104  28.354 -20.540    1.74    3.35   
4          12.0     23.8 -10.280  21.774 -34.111    1.72    3.56   

   type_confidence    vx0      vy0    vz0           x     x0           y  \
0              2.0 -6.409 -136.065 -3.995  101.140000  2.280  158.780000   
1              2.0 -8.411 -135.690 -5.980  124.280000  2.119  175.410000   
2              2.0 -9.802 -137.668 -3.337  136.740000  2.127  150.110000   
3              2.0 -8.071 -133.005 -6.567  109.685636  2.279  187.463482   
4              2.0 -6.309 -110.409  0.325  146.527525  2.179  177.242829   

     y0     z0  pfx_x  pfx_z  nasty  zone code type pitch_type  event_num  \
0  50.0  5.302   4.16  10.93   55.0   3.0    C    S         FF          3   
1  50.0  5.307   6.57  12.00   31.0   5.0    S    S         FF          4   
2  50.0  5.313   7.61  10.88   49.0   1.0    F    S         FF          5   
3  50.0  5.210   1.17   6.45   41.0  13.0    B    B         FF          6   
4  50.0  5.557  -8.43  -1.65   18.0  13.0    B    B         CU          7   

   b_score       ab_id  b_count  s_count  outs  pitch_num  on_1b  on_2b  \
0      0.0  2015000001      0.0      0.0   0.0        1.0    0.0    0.0   
1      0.0  2015000001      0.0      1.0   0.0        2.0    0.0    0.0   
2      0.0  2015000001      0.0      2.0   0.0        3.0    0.0    0.0   
3      0.0  2015000001      0.0      2.0   0.0        4.0    0.0    0.0   
4      0.0  2015000001      1.0      2.0   0.0        5.0    0.0    0.0   

   on_3b  batter_id      event       g_id  inning  o  p_score p_throws  \
0    0.0     572761  Groundout  201500001       1  1        0        L   
1    0.0     572761  Groundout  201500001       1  1        0        L   
2    0.0     572761  Groundout  201500001       1  1        0        L   
3    0.0     572761  Groundout  201500001       1  1        0        L   
4    0.0     572761  Groundout  201500001       1  1        0        L   

   pitcher_id stand   top  
0      452657     L  True  
1      452657     L  True  
2      452657     L  True  
3      452657     L  True  
4      452657     L  True

In [30]:
new_df1 = pd.merge(new_df, games,  how='left', left_on='g_id', right_on = 'g_id')
new_df1.head()

px     pz  start_speed  end_speed  spin_rate  spin_dir  break_angle  \
0  0.416  2.963         92.9       84.1   2305.052   159.235        -25.0   
1 -0.191  2.347         92.8       84.1   2689.935   151.402        -40.7   
2 -0.518  3.284         94.1       85.2   2647.972   145.125        -43.7   
3 -0.641  1.221         91.0       84.0   1289.590   169.751         -1.3   
4 -1.821  2.083         75.4       69.6   1374.569   280.671         18.4   

   break_length  break_y      ax      ay      az  sz_bot  sz_top  \
0           3.2     23.7   7.665  34.685 -11.960    1.72    3.56   
1           3.4     23.7  12.043  34.225 -10.085    1.72    3.56   
2           3.7     23.7  14.368  35.276 -11.560    1.72    3.56   
3           5.0     23.8   2.104  28.354 -20.540    1.74    3.35   
4          12.0     23.8 -10.280  21.774 -34.111    1.72    3.56   

   type_confidence    vx0      vy0    vz0           x     x0           y  \
0              2.0 -6.409 -136.065 -3.995  101.140000  2.280  158.780000   
1              2.0 -8.411 -135.690 -5.980  124.280000  2.119  175.410000   
2              2.0 -9.802 -137.668 -3.337  136.740000  2.127  150.110000   
3              2.0 -8.071 -133.005 -6.567  109.685636  2.279  187.463482   
4              2.0 -6.309 -110.409  0.325  146.527525  2.179  177.242829   

     y0     z0  pfx_x  pfx_z  nasty  zone code type pitch_type  event_num  \
0  50.0  5.302   4.16  10.93   55.0   3.0    C    S         FF          3   
1  50.0  5.307   6.57  12.00   31.0   5.0    S    S         FF          4   
2  50.0  5.313   7.61  10.88   49.0   1.0    F    S         FF          5   
3  50.0  5.210   1.17   6.45   41.0  13.0    B    B         FF          6   
4  50.0  5.557  -8.43  -1.65   18.0  13.0    B    B         CU          7   

   b_score       ab_id  b_count  s_count  outs  pitch_num  on_1b  on_2b  \
0      0.0  2015000001      0.0      0.0   0.0        1.0    0.0    0.0   
1      0.0  2015000001      0.0      1.0   0.0        2.0    0.0    0.0   
2      0.0  2015000001      0.0      2.0   0.0        3.0    0.0    0.0   
3      0.0  2015000001      0.0      2.0   0.0        4.0    0.0    0.0   
4      0.0  2015000001      1.0      2.0   0.0        5.0    0.0    0.0   

   on_3b  batter_id      event       g_id  inning  o  p_score p_throws  \
0    0.0     572761  Groundout  201500001       1  1        0        L   
1    0.0     572761  Groundout  201500001       1  1        0        L   
2    0.0     572761  Groundout  201500001       1  1        0        L   
3    0.0     572761  Groundout  201500001       1  1        0        L   
4    0.0     572761  Groundout  201500001       1  1        0        L   

   pitcher_id stand   top  attendance  away_final_score away_team        date  \
0      452657     L  True       35055                 3       sln  2015-04-05   
1      452657     L  True       35055                 3       sln  2015-04-05   
2      452657     L  True       35055                 3       sln  2015-04-05   
3      452657     L  True       35055                 3       sln  2015-04-05   
4      452657     L  True       35055                 3       sln  2015-04-05   

   elapsed_time  home_final_score home_team start_time    umpire_1B  \
0           184                 0       chn    7:17 PM  Mark Wegner   
1           184                 0       chn    7:17 PM  Mark Wegner   
2           184                 0       chn    7:17 PM  Mark Wegner   
3           184                 0       chn    7:17 PM  Mark Wegner   
4           184                 0       chn    7:17 PM  Mark Wegner   

      umpire_2B        umpire_3B     umpire_HP     venue_name  \
0  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
1  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
2  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
3  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
4  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   

             

In [31]:
new_df2 = pd.merge(new_df1, player_name,  how='left', left_on='batter_id', right_on = 'batter_id')
new_df2.head()

px     pz  start_speed  end_speed  spin_rate  spin_dir  break_angle  \
0  0.416  2.963         92.9       84.1   2305.052   159.235        -25.0   
1 -0.191  2.347         92.8       84.1   2689.935   151.402        -40.7   
2 -0.518  3.284         94.1       85.2   2647.972   145.125        -43.7   
3 -0.641  1.221         91.0       84.0   1289.590   169.751         -1.3   
4 -1.821  2.083         75.4       69.6   1374.569   280.671         18.4   

   break_length  break_y      ax      ay      az  sz_bot  sz_top  \
0           3.2     23.7   7.665  34.685 -11.960    1.72    3.56   
1           3.4     23.7  12.043  34.225 -10.085    1.72    3.56   
2           3.7     23.7  14.368  35.276 -11.560    1.72    3.56   
3           5.0     23.8   2.104  28.354 -20.540    1.74    3.35   
4          12.0     23.8 -10.280  21.774 -34.111    1.72    3.56   

   type_confidence    vx0      vy0    vz0           x     x0           y  \
0              2.0 -6.409 -136.065 -3.995  101.140000  2.280  158.780000   
1              2.0 -8.411 -135.690 -5.980  124.280000  2.119  175.410000   
2              2.0 -9.802 -137.668 -3.337  136.740000  2.127  150.110000   
3              2.0 -8.071 -133.005 -6.567  109.685636  2.279  187.463482   
4              2.0 -6.309 -110.409  0.325  146.527525  2.179  177.242829   

     y0     z0  pfx_x  pfx_z  nasty  zone code type pitch_type  event_num  \
0  50.0  5.302   4.16  10.93   55.0   3.0    C    S         FF          3   
1  50.0  5.307   6.57  12.00   31.0   5.0    S    S         FF          4   
2  50.0  5.313   7.61  10.88   49.0   1.0    F    S         FF          5   
3  50.0  5.210   1.17   6.45   41.0  13.0    B    B         FF          6   
4  50.0  5.557  -8.43  -1.65   18.0  13.0    B    B         CU          7   

   b_score       ab_id  b_count  s_count  outs  pitch_num  on_1b  on_2b  \
0      0.0  2015000001      0.0      0.0   0.0        1.0    0.0    0.0   
1      0.0  2015000001      0.0      1.0   0.0        2.0    0.0    0.0   
2      0.0  2015000001      0.0      2.0   0.0        3.0    0.0    0.0   
3      0.0  2015000001      0.0      2.0   0.0        4.0    0.0    0.0   
4      0.0  2015000001      1.0      2.0   0.0        5.0    0.0    0.0   

   on_3b  batter_id      event       g_id  inning  o  p_score p_throws  \
0    0.0     572761  Groundout  201500001       1  1        0        L   
1    0.0     572761  Groundout  201500001       1  1        0        L   
2    0.0     572761  Groundout  201500001       1  1        0        L   
3    0.0     572761  Groundout  201500001       1  1        0        L   
4    0.0     572761  Groundout  201500001       1  1        0        L   

   pitcher_id stand   top  attendance  away_final_score away_team        date  \
0      452657     L  True       35055                 3       sln  2015-04-05   
1      452657     L  True       35055                 3       sln  2015-04-05   
2      452657     L  True       35055                 3       sln  2015-04-05   
3      452657     L  True       35055                 3       sln  2015-04-05   
4      452657     L  True       35055                 3       sln  2015-04-05   

   elapsed_time  home_final_score home_team start_time    umpire_1B  \
0           184                 0       chn    7:17 PM  Mark Wegner   
1           184                 0       chn    7:17 PM  Mark Wegner   
2           184                 0       chn    7:17 PM  Mark Wegner   
3           184                 0       chn    7:17 PM  Mark Wegner   
4           184                 0       chn    7:17 PM  Mark Wegner   

      umpire_2B        umpire_3B     umpire_HP     venue_name  \
0  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
1  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
2  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
3  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
4  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   

             

In [32]:
new_df2['Batters Name'] = new_df2[['first_name', 'last_name']].apply(lambda x: ' '.join(x), axis=1)

In [33]:
new_df2.drop(['first_name', 'last_name'], axis=1, inplace=True)

In [34]:
player_name.rename(columns={'batter_id': 'pitcher_id'}, inplace=True)

In [36]:
final_df = pd.merge(new_df2, player_name,  how='left', left_on='pitcher_id', right_on = 'pitcher_id')
final_df.head()

px     pz  start_speed  end_speed  spin_rate  spin_dir  break_angle  \
0  0.416  2.963         92.9       84.1   2305.052   159.235        -25.0   
1 -0.191  2.347         92.8       84.1   2689.935   151.402        -40.7   
2 -0.518  3.284         94.1       85.2   2647.972   145.125        -43.7   
3 -0.641  1.221         91.0       84.0   1289.590   169.751         -1.3   
4 -1.821  2.083         75.4       69.6   1374.569   280.671         18.4   

   break_length  break_y      ax      ay      az  sz_bot  sz_top  \
0           3.2     23.7   7.665  34.685 -11.960    1.72    3.56   
1           3.4     23.7  12.043  34.225 -10.085    1.72    3.56   
2           3.7     23.7  14.368  35.276 -11.560    1.72    3.56   
3           5.0     23.8   2.104  28.354 -20.540    1.74    3.35   
4          12.0     23.8 -10.280  21.774 -34.111    1.72    3.56   

   type_confidence    vx0      vy0    vz0           x     x0           y  \
0              2.0 -6.409 -136.065 -3.995  101.140000  2.280  158.780000   
1              2.0 -8.411 -135.690 -5.980  124.280000  2.119  175.410000   
2              2.0 -9.802 -137.668 -3.337  136.740000  2.127  150.110000   
3              2.0 -8.071 -133.005 -6.567  109.685636  2.279  187.463482   
4              2.0 -6.309 -110.409  0.325  146.527525  2.179  177.242829   

     y0     z0  pfx_x  pfx_z  nasty  zone code type pitch_type  event_num  \
0  50.0  5.302   4.16  10.93   55.0   3.0    C    S         FF          3   
1  50.0  5.307   6.57  12.00   31.0   5.0    S    S         FF          4   
2  50.0  5.313   7.61  10.88   49.0   1.0    F    S         FF          5   
3  50.0  5.210   1.17   6.45   41.0  13.0    B    B         FF          6   
4  50.0  5.557  -8.43  -1.65   18.0  13.0    B    B         CU          7   

   b_score       ab_id  b_count  s_count  outs  pitch_num  on_1b  on_2b  \
0      0.0  2015000001      0.0      0.0   0.0        1.0    0.0    0.0   
1      0.0  2015000001      0.0      1.0   0.0        2.0    0.0    0.0   
2      0.0  2015000001      0.0      2.0   0.0        3.0    0.0    0.0   
3      0.0  2015000001      0.0      2.0   0.0        4.0    0.0    0.0   
4      0.0  2015000001      1.0      2.0   0.0        5.0    0.0    0.0   

   on_3b  batter_id      event       g_id  inning  o  p_score p_throws  \
0    0.0     572761  Groundout  201500001       1  1        0        L   
1    0.0     572761  Groundout  201500001       1  1        0        L   
2    0.0     572761  Groundout  201500001       1  1        0        L   
3    0.0     572761  Groundout  201500001       1  1        0        L   
4    0.0     572761  Groundout  201500001       1  1        0        L   

   pitcher_id stand   top  attendance  away_final_score away_team        date  \
0      452657     L  True       35055                 3       sln  2015-04-05   
1      452657     L  True       35055                 3       sln  2015-04-05   
2      452657     L  True       35055                 3       sln  2015-04-05   
3      452657     L  True       35055                 3       sln  2015-04-05   
4      452657     L  True       35055                 3       sln  2015-04-05   

   elapsed_time  home_final_score home_team start_time    umpire_1B  \
0           184                 0       chn    7:17 PM  Mark Wegner   
1           184                 0       chn    7:17 PM  Mark Wegner   
2           184                 0       chn    7:17 PM  Mark Wegner   
3           184                 0       chn    7:17 PM  Mark Wegner   
4           184                 0       chn    7:17 PM  Mark Wegner   

      umpire_2B        umpire_3B     umpire_HP     venue_name  \
0  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
1  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
2  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
3  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   
4  Marty Foster  Mike Muchlinski  Mike Winters  Wrigley Field   

             

In [38]:
final_df['Pitchers Name'] = final_df[['first_name', 'last_name']].apply(lambda x: ' '.join(x), axis=1)